<a href="https://colab.research.google.com/github/Bantwini/CODESOFT/blob/main/Titanic_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'titanic-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1818188%2F2965537%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T102558Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D11b539bb666f1fee0ab1d9e1331abbaf9a10c147fa5bb62e50cf43d2e8273f8deaf643c5d8eb1fb20fb5b29b819e2fead1bd2dd2c85bba92b9cabc3b7385c9107b35e29e74cb648391551e5ed4ab5ec4064b78430daab23aebfe5641d7932931e98077e10eb1a03ff49015d34581b316478e1101118a461b8e9a551917660c42a9d57609863cc34ae084be8234e6cbbef1d7b62d508724db9d82c5fabc50cc27d2fce36e0483967ab15e30ae768bcf1a431dada64790c058099257757a3ac6b00f6093b92b104d4c21b90f99a33db80d5a505d40765593a440eae617c59d385da267c40637c3ba9ac9c3d74971f4315ad2fd00cc5fba53e67196bffe90a76b90'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 22564 bytes downloaded
Downloaded and uncompressed: titanic-dataset
Data source import complete.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df= pd.read_csv("/kaggle/input/titanic-dataset/Titanic-Dataset.csv")

Path of the file to read

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [ ]:
Survived = df['Survived'].value_counts().reset_index()

In [ ]:
Survived

In [ ]:
data = {'Survived': ['Male - No', 'Male - Yes', 'Female - No', 'Female - Yes'],
        'Counts': [100, 50, 30, 80]}
Survived = pd.DataFrame(data)

plt.figure(figsize=(8, 6))
plt.bar(Survived['Survived'], Survived['Counts'],color=["Green","Brown","Blue","Purple"])
plt.xticks(Survived['Survived'])
plt.title('Survival Prediction')
plt.xlabel('Gender and Survival ')
plt.ylabel('Number of People')
plt.show()

In [ ]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

In [ ]:
inputs = df.drop('Survived',axis='columns')
target = df['Survived']

In [ ]:
target

In [ ]:
sex=pd.get_dummies(inputs.Sex)
sex.head()

In [ ]:
inputs=pd.concat([inputs,sex],axis="columns")
inputs.head()

In [ ]:
inputs.drop(["Sex"],axis="columns" ,inplace=True)

In [ ]:
inputs.head()

In [ ]:
inputs.isna().sum()

In [ ]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

In [ ]:
inputs.info()

In [ ]:
inputs.isna().sum()

In [ ]:
counts = df.groupby(['Survived', 'Sex']). size().unstack().fillna(0)

#This determines the width of the bar
bar_width = 0.35
index = counts.index

fig, ax = plt.subplots()

#Plot bars for each gender
bar1 = ax.bar(index - bar_width/2, counts['male'], bar_width, label='male')
bar2 = ax.bar(index + bar_width/2, counts['female'], bar_width, label='female')


#Setting Labels and tittle
ax.set_xlabel('Survived')
ax.set_ylabel('Count')
ax.set_title('Survival Count by Gender')
ax.set_xticks(index)
ax.set_xticklabels(['Not Survived', 'Survived'])
ax.legend()

#Displaying the plot
plt.show()



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
inputs.corr()

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(inputs.corr(), annot=True, cmap='cool', fmt= ".2f")

In [ ]:
model=RandomForestClassifier()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

In [ ]:
pre=model.predict(X_test)

In [ ]:
matrices=r2_score(pre,y_test)
matrices